# Advent of Code 2022

## Day 17: Pyroclastic Flow

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2022.

My notebooks are getting messier.

Today didn't take me quite as long as yesterday, but still took a while.

Part 1 I just simulate the tetris blocks falling. I used a bit more classes and abstraction today. Key to not getting confused is splitting the `RockShape` class from the `FallingRock` class. As a small optimisation, the stack tracks all landed squares in the pile, but instead of using a set of points, which I started off doing, I use another class `ChamberPoints` which looks up the row number in a dict, and gets a list of 7 bools to check. Bonus rending an emoji stack of blocks.

Part 2 luckily didn't take too long to figure out the key, the number of units added to the stack each time is either 0, 1, 2, or 3, so start run the simulation for a large number of blocks, say 10000, then taking difference in heights giving 9999 differences, or 'deltas' and search for the period. I don't know if there's a good way to search for a period other than the dumb method I implemented, but in my case it comes out to be 80 not repeated deltas at for the first 80 drops, then a period of 1740 from then on. I grab the first 80 deltas, the next 1740 deltas, create a function that spits out another function that uses some modular arithmetic, partial sums, and multiplication to look up height of the stack after N blocks have dropped, in constant time w.r.t. N. Final code doesn't take long to run although the period finding is a little slow.

### Imports

In [ ]:
from enum import Enum
from typing import Iterator, Optional
from collections import namedtuple
import tqdm.notebook as tqdm
from collections import defaultdict

### Directions

In [ ]:
class Direction(Enum):
    LEFT = 1
    RIGHT = 2
    UP = 3
    DOWN = 4

    def __str__(self):
        assert self in {Direction.LEFT, Direction.RIGHT, Direction.UP, Direction.DOWN}
        if self == Direction.LEFT:
            return '←'
        if self == Direction.RIGHT:
            return '→'
        if self == Direction.UP:
            return '↑'
        return '↓'

    def velocity(self):
        assert self in {Direction.LEFT, Direction.RIGHT, Direction.UP, Direction.DOWN}
        if self == Direction.LEFT:
            return Point(x=-1, y=0)
        if self == Direction.RIGHT:
            return Point(x=1, y=0)
        if self == Direction.UP:
            return Point(x=0, y=1)
        return Point(x=0, y=-1)

    def __repr__(self):
        return str(self)

    @staticmethod
    def parse(symbol: str) -> 'Direction':
        assert symbol in {'<', '>'}
        if symbol == '<':
            return Direction.LEFT
        return Direction.RIGHT

INPUT_FILE = 'data/input17.txt'

with open(INPUT_FILE) as file:
    line = next(file).strip()
    DIRECTIONS = [Direction.parse(symbol) for symbol in line]

def infinite_direction_source(*, limit: Optional[int] = None) -> Iterator[Direction]:
    global DIRECTIONS
    assert limit is None or (type(limit) == int and limit >= 0)
    count = 0
    while True:
        for rv in DIRECTIONS:
            if limit is not None and count >= limit:
                return
            yield rv
            count += 1

### Rocks Shapes

In [ ]:
class RockShape:

    __slots__ = ['slices']

    def __init__(self, encoding: list[str]):
        assert (len(encoding) >= 1), f'{len(encoding) = }'
        length = None
        slices = []
        for line in encoding:
            assert (len(line) >= 1), f'encoding has empty line'
            if length is None:
                length = len(line)
            else:
                assert (len(line) == length), f'encoding is not rectangular'
            for char in line:
                assert (char in {'.', '#'}), f'unknown character in encoding'
            slices.append(tuple([char == '#' for char in line]))
        self.slices = tuple(slices)

    def __str__(self):
        rv = ''
        for line in self.slices:
            for filled in line:
                if filled:
                    rv += '@'
                else:
                    rv += ' '
            rv += '\n'
        return rv

    def __repr__(self):
        return str(self)

ROCK_SHAPES = [
    RockShape(['####']),              # minus
    RockShape(['.#.', '###', '.#.']), # plus
    RockShape(['..#', '..#', '###']), # J
    RockShape(['#', '#', '#', '#']),  # I
    RockShape(['##', '##'])           # square
]

def infinite_rock_shape_source(*, limit: Optional[int] = None) -> Iterator[RockShape]:
    global ROCK_SHAPES
    assert limit is None or (type(limit) == int and limit >= 0)
    count = 0
    while True:
        for rv in ROCK_SHAPES:
            if limit is not None and count >= limit:
                return
            yield rv
            count += 1

### Falling Rocks

In [ ]:
# noinspection PyTypeChecker
Point = namedtuple('Point', ['x', 'y'])

def add(a: Point, b: Point) -> Point:
    return Point(x=a.x+b.x, y=a.y+b.y)

def negate(a: Point) -> Point:
    return Point(x=-a.x, y=-a.y)

In [ ]:
class FallingRock:

    __slots__ = ['shape', 'bottom_left']

    def __init__(self, shape: RockShape, bottom_left: Point):
        self.shape = shape
        self.bottom_left = bottom_left

    def to_points(self) -> set[Point]:
        rv = []
        current_y = self.bottom_left.y
        for neg_y_offset, cross_section in reversed(list(enumerate(self.shape.slices))):
            current_x = self.bottom_left.x
            for x_offset, is_rock in enumerate(cross_section):
                if is_rock:
                    rv.append(Point(x=current_x, y=current_y))
                current_x += 1
            current_y += 1
        return set(rv)

    def attempt_move(self, direction: Direction, chamber: 'Chamber'):

        velocity = direction.velocity()

        self.bottom_left = add(self.bottom_left, velocity)
        points = self.to_points()
        crashed = False

        # check for going outside area
        for p in points:
            if p.y < 0 or p.x < 0 or p.x >= Chamber.WIDTH:
                crashed = True
                break

        # check for landing
        if not crashed:
            for p in points:
                if p in chamber.points:
                    crashed = True
                    break

        # move back
        if crashed:
            self.bottom_left = add(self.bottom_left, negate(velocity))
            #points = self.to_points()
            #for p in points:
            #    assert (p not in chamber.points), f'moved rock back, but {p} is landed'
            return False

        return True

### Chamber

In [ ]:
class ChamberPoints:

    __slots__ = ['rows']

    def __init__(self):
        self.rows = defaultdict(lambda: [False] * Chamber.WIDTH)

    def add(self, point: Point):
        row = self.rows[point.y]
        row[point.x] = True

    def __contains__(self, point: Point):
        return self.rows[point.y][point.x]

In [ ]:
class Chamber:

    WIDTH = 7

    __slots__ = ['points', 'rocks', 'stack_height']

    def __init__(self):
        self.points = ChamberPoints()
        self.rocks: set[FallingRock] = set()
        self.stack_height = 0


    def land(self, falling_rock: FallingRock) -> None:
        for p in falling_rock.to_points():
            assert (p not in self.points), f'trying to land rock but aleady landed point {p}'
            self.points.add(p)
            self.rocks.add(falling_rock)
            if p.y + 1 > self.stack_height:
                self.stack_height = p.y + 1

    def spawn_rock(self, shape: RockShape) -> FallingRock:
        start = Point(x=2, y=3+self.stack_height)
        return FallingRock(shape, start)

    def render(self, falling_rock: Optional[FallingRock] = None) -> None:
        points = falling_rock.to_points() if falling_rock else set()
        start_height = self.stack_height
        if falling_rock is not None:
            new_top = falling_rock.bottom_left.y + len(falling_rock.shape.slices)
            if new_top > start_height:
                start_height = new_top
        for y in range(start_height, -1, -1):
            print('🔳', end='')
            for x in range(0, Chamber.WIDTH):
                p = Point(x=x, y=y)
                if p in self.points:
                    print('🟥', end='')
                elif p in points:
                    print('🟩', end='')
                else:
                    print('⬛', end='')
            print(f'🔳 {y}')
        print(f'🔳🔳🔳🔳🔳🔳🔳🔳🔳\tstack height = {self.stack_height}')
        print()

### Part 1

In [ ]:
def main():

    chamber = Chamber()

    directions = infinite_direction_source()

    verbose = False
    print_each_block = False
    print_end = True

    num_blocks = 2022

    for shape in tqdm.tqdm(infinite_rock_shape_source(limit=num_blocks), total=num_blocks):

        rock: Optional[FallingRock] = chamber.spawn_rock(shape)
        if verbose:
            print('rock begins falling')
            chamber.render(rock)

        while rock is not None:

            direction = next(directions)
            if verbose:
                print(f'jet of gas pushes {direction}', end= '')
            moved = rock.attempt_move(direction, chamber)
            if verbose:
                if not moved:
                    print(' but nothing happened', end='')
                print(':')
                chamber.render(rock)

            direction = Direction.DOWN
            if verbose:
                print('rock falls 1 unit', end= '')
            moved = rock.attempt_move(direction, chamber)
            if not moved:
                if verbose:
                    print(' causing it to come to a rest', end='')
                chamber.land(rock)
                rock = None
            if verbose:
                print(':')
            if verbose or print_each_block:
                chamber.render(rock)

    print('Simulation ended.')
    print(f'stack height is {chamber.stack_height}')
    print()
    if not verbose and not print_each_block:
        if print_end:
            chamber.render()

In [ ]:
if __name__ == '__main__':
    main()

### Part 2

In [ ]:
chamber = Chamber()
directions = infinite_direction_source()
num_blocks = 10000
heights = [0]
for shape in tqdm.tqdm(infinite_rock_shape_source(limit=num_blocks), total=num_blocks):
    rock: Optional[FallingRock] = chamber.spawn_rock(shape)
    while rock is not None:
        direction = next(directions)
        rock.attempt_move(direction, chamber)
        direction = Direction.DOWN
        moved = rock.attempt_move(direction, chamber)
        if not moved:
            chamber.land(rock)
            rock = None
    heights.append(chamber.stack_height)

deltas = []
for i in range(1, len(heights)):
    deltas.append(heights[i] - heights[i-1])

In [ ]:
deltas[:10]

In [ ]:
def is_potential_period(seq, period, required):
    expected = None
    checked = 0
    for start in range(0, len(seq), period):
        current = seq[start:start+period]
        if expected is None:
            expected = current
        elif len(current) < len(expected):
            return True
        elif current != expected:
            return False
        else:
            # current == expected
            checked += 1
            if checked >= required:
                return True
    return True

def find_period(seq, required):
    maximum = len(seq) // 2
    for p in range(1, maximum+1):
        if is_potential_period(seq, p, required):
            yield p

def loop_period(seq, required):
    for stripped in tqdm.tqdm(range(len(seq))):
        for potential in find_period(seq, required):
            print(f'Strip off the first {stripped} elements.')
            print(f'Then, period seems to be {potential}')
            return
        seq = seq[1:]

loop_period(deltas, required=3)

### Output Saved

    Strip off the first 80 elements.
    Then, period seems to be 1740

### Testing Function Reconstruction with the example

In [ ]:
example_deltas = [1, 3, 3, 1, 3, 3, 7, 1, 3, 3, 7, 1, 3, 3, 7, 1, 3, 3, 7, 1, 3, 3, 7]
example_values = [0]
for d in example_deltas:
    example_values.append(example_values[-1] + d)
print(example_values)

example_deltas_recomputed = []
for i in range(1, len(example_values)):
    example_deltas_recomputed.append(example_values[i] - example_values[i-1])

print(f'{example_deltas = }')
print(f'{example_deltas_recomputed = }')
assert example_deltas == example_deltas_recomputed
print('recomputed correctly')

loop_period(example_deltas, required=3)

In [ ]:
header_length = 3
loop_part_length = 4

In [ ]:
header = example_deltas[: header_length]
loop1 = example_deltas[header_length : header_length+loop_part_length]
loop2 = example_deltas[header_length+loop_part_length : header_length+loop_part_length*2]
loop3 = example_deltas[header_length+loop_part_length*2 : header_length+loop_part_length*3]

print(header)
print()
print(loop1)
print()
print(loop1 == loop2)
print(loop2 == loop3)

In [ ]:
def reconstruct_function(header, loop):
    sum_header = sum(header)
    sum_loop = sum(loop)
    def f(x):
        assert type(x) == int
        assert x >= 0
        if x <= len(header):
            return sum([d for d in header[:x]])
        rv = sum_header + sum_loop * ((x - len(header)) // len(loop))
        if (x - len(header)) % len(loop) == 0:
            return rv
        else:
            return rv + sum(loop[:(x - len(header)) % len(loop)])
    return f

In [ ]:
reconstructed = reconstruct_function(header, loop1)
for i in range(len(example_values)):
    print(i, reconstructed(i), example_values[i], reconstructed(i) == example_values[i], sep='\t')

### Now back to real data

In [ ]:
header_length = 80
loop_part_length = 1740

header = deltas[: header_length]
loop1 = deltas[header_length : header_length+loop_part_length]
loop2 = deltas[header_length+loop_part_length : header_length+loop_part_length*2]
loop3 = deltas[header_length+loop_part_length*2 : header_length+loop_part_length*3]

print(header)
print()
print(loop1)
print()
print(loop1 == loop2)
print(loop2 == loop3)

In [ ]:
print('checking header add up to next')
print(f'{sum(header) = }')
print(f'{heights[header_length] = }')
assert sum(header) == heights[header_length]
print()
offset_height = heights[header_length]
print(f'must add {offset_height} to the height')

In [ ]:
reconstructed = reconstruct_function(header, loop1)

for i, h in enumerate(heights):
    assert reconstructed(i) == h

print('okay for all heights!')

### Part 1 Answer (Again)

In [ ]:
# checking answer for part 1
reconstructed(2022)

### Part 2 Answer

In [ ]:
reconstructed(1000000000000)